In [1]:
from appgeopy import *
from my_packages import *

In [9]:
# fpath = r"D:\VINHTRUONG\MYTOOLBOX\MultiLayerCompactionMonitoringWells\MLCW_Data_Extraction\YL_WSYL23G1_TUKU_土庫.xlsx"

# Define the main folder path
data_folder = r"D:\1000_SCRIPTS\001_PreQE_Scripts\MultiLayerCompactionMonitoringWells\MLCW_Data_Extraction"

# Get list of all Excel files in the folder
excel_files = glob(os.path.join(data_folder, "*.xlsx"))

# Initialize a dictionary to store the results
results = cache = {
    "WellCode": [],
    "Ename": [],
    "Cname": [],
    "Total_FineGrain_Thickness": [],
    "BottomDepth": [],
    "Percentage": [],
}

# Define the threshold for bottom depth
depth_threshold = 350

# Process each file
for file_path in tqdm(excel_files[:]):
    # Extract file name information
    file_name = os.path.basename(file_path).split(".")[0]
    _, well_code, english_name, chinese_name = file_name.split("_")

    # Load data from the Excel file
    compaction_data = pd.read_excel(file_path)

    # Filter data based on depth and soil category criteria
    filter_condition_by_depth = "TOP>=0 & BOTTOM <= @depth_threshold"
    filter_condition_by_material = "SOIL_CATEGORY in [5, 6]"

    filtered_data_by_depth = compaction_data.query(filter_condition_by_depth)
    bottom_depth = filtered_data_by_depth.BOTTOM.iloc[-1]

    # Calculate the sum of fine-grain soil thickness and bottom depth
    filtered_data_by_material = filtered_data_by_depth.query(
        filter_condition_by_material
    )
    total_finegrain_thickness = filtered_data_by_material.THICKNESS.sum()

    # Calculate the percentage of fine-grain soil thickness
    finegrain_percentage = np.round(
        total_finegrain_thickness * 100 / bottom_depth, 2
    )

    # Append results to the dictionary
    for key, value in zip(
        results.keys(),
        [
            well_code,
            english_name,
            chinese_name,
            total_finegrain_thickness,
            bottom_depth,
            finegrain_percentage,
        ],
    ):
        results[key].append(value)

# Convert the results dictionary to a DataFrame
finegrain_thickness_summary = pd.DataFrame(results)

  0%|          | 0/32 [00:00<?, ?it/s]

In [11]:
show(finegrain_thickness_summary)

Loading ITables v2.4.5 from the internet... (need help?)
